# MLDM Group Assignment
### Members: Brigitte Aznar, Daniel Kostic and Stefan Vujovic

Data explanation
survival        Survival
                (0 = No; 1 = Yes)
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data.csv', index_col=0)
# df.survived = df.survived.apply(lambda x: "yes" if x else "no")
# df.to_csv('data1.csv')

In [2]:
# df['sex1'] = df.sex.apply(lambda x: 1 if x=="male" else 0)
# df['survived'] = df.survived.apply(lambda x: "survived" if x==1 else "")
def makeClass(x):
    if x==1: return "firstClass"
    elif x==2: return "secondClass"
    else: return "thirdClass"
df['pclass'] = df.pclass.apply(lambda x: makeClass(x))
df.fare = df.fare.apply(lambda x: 33 if pd.isnull(x) else x)
# df.fare.dropna(inplace=True, how='any')
# df.embarked.dropna(inplace=True)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,1,firstClass,female,29.0000,0,0,211.3375,S
1,1,firstClass,male,0.9167,1,2,151.5500,S
2,0,firstClass,female,2.0000,1,2,151.5500,S
3,0,firstClass,male,30.0000,1,2,151.5500,S
4,0,firstClass,female,25.0000,1,2,151.5500,S


In [3]:
df.embarked.isnull().value_counts()

False    1307
True        2
Name: embarked, dtype: int64

In [4]:
df.age.describe()
mean = 29.881135
std_dev = 14.413500
age_fillers = np.random.normal(mean, std_dev, 280)
age_fillers = [i for i in age_fillers if i>0][:263]
df['age1'] = df.age.apply(lambda x: age_fillers.pop() if np.isnan(x) else x)

In [5]:
cat_columns = ['pclass', 'sex'] # embarked
for cat_col in cat_columns:
    df = df.join(pd.get_dummies(df[cat_col]))

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
from sklearn import tree
features = ['age1', 'firstClass', 'secondClass', 'thirdClass', 'sibsp', 'parch', 'fare', 'female'] #, 'C', 'Q', 'S'
clf = tree.DecisionTreeClassifier(criterion='gini')
clf = clf.fit(df[features], df['survived'])

In [8]:
tree.export_graphviz(clf, out_file='tree.dot')

In [9]:
for feat in features:
    print("{}, from {}".format(df[feat].isnull().sum(), feat))

0, from age1
0, from firstClass
0, from secondClass
0, from thirdClass
0, from sibsp
0, from parch
0, from fare
0, from female


In [10]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,age1,firstClass,secondClass,thirdClass,female,male
0,1,firstClass,female,29.0000,0,0,211.3375,S,29.0000,1,0,0,1,0
1,1,firstClass,male,0.9167,1,2,151.5500,S,0.9167,1,0,0,0,1
2,0,firstClass,female,2.0000,1,2,151.5500,S,2.0000,1,0,0,1,0
3,0,firstClass,male,30.0000,1,2,151.5500,S,30.0000,1,0,0,0,1
4,0,firstClass,female,25.0000,1,2,151.5500,S,25.0000,1,0,0,1,0


In [11]:
def gini(attribute):
    att_values = df[attribute].unique()
    for class_val in [0,1]:
        for val in att_values:
            size = df[df[attribute]==val].shape[0]
            gini = df[df[attribute]==val].survived.count() / float(size)
            print ("Class:{}, Attribute:{}, GINI:{}".format(class_val, val, gini))

In [12]:
gini('sex')

Class:0, Attribute:female, GINI:1.0
Class:0, Attribute:male, GINI:1.0
Class:1, Attribute:female, GINI:1.0
Class:1, Attribute:male, GINI:1.0


In [13]:
[1,2,3,3,3,3,3].count(3)

5

In [14]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,age1,firstClass,secondClass,thirdClass,female,male
0,1,firstClass,female,29.0000,0,0,211.3375,S,29.0000,1,0,0,1,0
1,1,firstClass,male,0.9167,1,2,151.5500,S,0.9167,1,0,0,0,1
2,0,firstClass,female,2.0000,1,2,151.5500,S,2.0000,1,0,0,1,0
3,0,firstClass,male,30.0000,1,2,151.5500,S,30.0000,1,0,0,0,1
4,0,firstClass,female,25.0000,1,2,151.5500,S,25.0000,1,0,0,1,0


In [15]:
from DecisionTree import DecisionTree as Dt

In [16]:
dt = Dt()
dt.fit(df[['sex', 'age1', 'pclass', 'fare']], df['survived'])

D:\dev\machine-learning-data-mining\assignment_4\DecisionTree.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  features[label] = target.values


In [17]:
predictions = dt.predict(df[['sex', 'age1', 'pclass', 'fare']])

In [18]:
df['predictions'] = pd.Series(predictions)
df['diff'] = df['survived'] - df['predictions']
df['diff'].sum()

27

In [21]:
df[['survived', 'predictions', 'diff']].head()

,survived,predictions,diff
0,1,1,0
1,1,1,0
2,0,0,0
3,0,0,0
4,0,0,0


In [31]:
from DecisionTree import bagging
from sklearn.model_selection import train_test_split
from time import time
now = time()
x_train, x_test, y_train, y_test = train_test_split(df.drop('survived', 1), df['survived'], test_size=0.2)
x_train['survived']  = y_train
x_test['survived'] = y_test

preds = bagging(x_train, x_test, ['sex', 'age1', 'pclass', 'fare'], 'survived', 9)
print(now - time())

D:\dev\machine-learning-data-mining\assignment_4\DecisionTree.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  features[label] = target.values


-170.78260207176208


In [32]:
print(len(x_test))
print(len(y_test))
print(len(preds))

262
262
524


In [33]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))

ValueError: Found input variables with inconsistent numbers of samples: [262, 524]